In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('data/adult_one_hot.csv')
X_np = df.iloc[:,:-1].as_matrix()
X_np = (X_np - np.mean(X_np,axis=0)) / np.std(X_np,axis=0)
Y_np = np.expand_dims(df.iloc[:,-1].as_matrix(),1).astype(int)

## Logistic Regression

In [45]:
num_features = df.shape[1]-1
learning_rate = 0.1
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.zeros([num_features, 1])) # the variables will be all zeros initially
b = tf.Variable(tf.zeros([1]))

In [47]:
Yhat = tf.sigmoid(X @ W + b)
loss               = tf.reduce_mean(-(Y*tf.log(Yhat) + (1-Y)*tf.log(1-Yhat)))
correct_prediction = tf.equal(tf.cast(Yhat > 0.5, tf.float32), Y)           
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
optimizer = tf.train.GradientDescentOptimizer(learning_rate) 
update = optimizer.minimize(loss) 

In [48]:
num_epochs  = 400

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        _, loss_val, acc_val = sess.run([update, loss, accuracy], 
                                         feed_dict={X: X_np, 
                                                    Y: Y_np})
        if epoch % 100 == 0:
            print("Epoch:",epoch,"Cost:",loss_val,"Accuracy:",acc_val)        
#     print("Test Accuracy:", accuracy.eval({X: X_np, Y: mnist.test.labels}))

Epoch: 0 Cost: 0.69319 Accuracy: 0.75919
Epoch: 100 Cost: 0.357691 Accuracy: 0.843494
Epoch: 200 Cost: 0.336354 Accuracy: 0.847087
Epoch: 300 Cost: 0.329218 Accuracy: 0.848008


## Logisitic Regression with Matching Regularizer

In [3]:
matching = pd.read_csv('data/adult_matches_nondesc.csv').astype(int)

In [9]:
num_features = df.shape[1]-1
learning_rate = 0.1
lambda_ = 0.001
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, 1])
Mid = tf.constant(matching['male.ids'])
Fid = tf.constant(matching['female.ids'])
W = tf.Variable(tf.zeros([num_features, 1])) # the variables will be all zeros initially
b = tf.Variable(tf.zeros([1]))

In [10]:
Yhat = tf.sigmoid(X @ W + b)
loss = tf.reduce_mean(-(Y*tf.log(Yhat) + (1-Y)*tf.log(1-Yhat)))
Mpred = tf.gather(X,Mid,axis=0) @ W + b
Fpred = tf.gather(X,Fid,axis=0) @ W + b
regularizer = tf.reduce_mean((Mpred - Fpred)**2)
objective = loss + lambda_ * regularizer
correct_prediction = tf.equal(tf.cast(Yhat > 0.5, tf.float32), Y)           
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
optimizer = tf.train.GradientDescentOptimizer(learning_rate) 
update = optimizer.minimize(objective) 

In [11]:
num_epochs  = 400

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        _, loss_val, acc_val = sess.run([update, loss, accuracy], 
                                         feed_dict={X: X_np, 
                                                    Y: Y_np})
        if epoch % 100 == 0:
            print("Epoch:",epoch,"Cost:",loss_val,"Accuracy:",acc_val)        
#     print("Test Accuracy:", accuracy.eval({X: X_np, Y: mnist.test.labels}))

Epoch: 0 Cost: 0.69319 Accuracy: 0.75919
Epoch: 100 Cost: 0.35877 Accuracy: 0.843586
Epoch: 200 Cost: 0.33789 Accuracy: 0.846319
Epoch: 300 Cost: 0.331058 Accuracy: 0.847947
